In [1]:
import os
import pandas as pd

data_df = pd.DataFrame(columns=['text','Intent'])
filenames = list()
intervention_subgraphs = list()
for directory, subdirectories, files in os.walk('C:/Users/Tamir/Desktop/HSE_Project/excel_files/'):
    for file in files:
        filenames.append(os.path.join(directory,file))
for filename in filenames:
    if (filename[len(filename) - 22] == '~'):
        continue
    filename = filename.replace(" ", " ")
    post_df = pd.read_excel(filename)
    
        #fix to add columns if they are not in the xlsx file
    if 'attachments' not in post_df.columns:
        post_df['attachments'] = ''
    if 'reply_to_comment' not in post_df.columns:
        post_df['reply_to_comment'] = 0
    if 'reply_to_user' not in post_df.columns:
        post_df['reply_to_user'] = 0
        
    flex_df = post_df.drop(columns=['attachments','date','from_id','id','reply_to_comment','reply_to_user','Content','Intervention'])
    data_df = data_df.append(flex_df,ignore_index = True)

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
tokenizer = RegexpTokenizer(r'\w+')
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def clean_text(text, remove_stopwords=True):
    """
    Removes stop words from the given text
    :param text: string with the text
    :param remove_stopwords: flag to enable removal of stop words
    :return: set of tokens without any stop words
    """
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    tags = list()
    

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('russian'))
        tokens = [w for w in tokens if w not in stops]
    for i in range(len(tokens)):
        tokens[i] = morph.parse(tokens[i])[0].normal_form
        t = morph.parse(tokens[i])[0]
        tokens[i] = tokens[i] + '_' + str(t.tag.POS)
    return tokens

delete_index = list()

for index,row in data_df.iterrows():
    if ( pd.isnull(data_df['text'].loc[index]) or pd.isnull(data_df['Intent'].loc[index]) 
              or (data_df['Intent'].loc[index] == ' -' ) ):
        delete_index.append(index)        
    else:
        data_df['text'].loc[index] = clean_text(data_df['text'].loc[index])

data_df = data_df.drop(index = delete_index)
data_df = data_df.reset_index(drop=True)

for index,row in data_df.iterrows():
    data_df['Intent'].loc[index] = data_df['Intent'].loc[index][0]
    if(data_df['Intent'].loc[index] == 'E'):
        print('change in ' + str(index))
        data_df['Intent'].loc[index] = 'Е'
        

            

change in 3158


In [3]:
#import json
#pymorphy_tags={
#        +'NOUN',  # имя существительное
#        +'ADJF',  # имя прилагательное (полное)
#        +'ADJS',  # имя прилагательное (краткое)
#        +'VERB',  # глагол (личная форма)
#        +'INFN',  # глагол (инфинитив)
#        +'PRTF',  # причастие (полное)
#        +'PRTS',  # причастие (краткое)
#        +'GRND',  # деепричастие
#        +'NUMR',  # числительное
#        +'ADVB',  # наречие
#        +'NPRO',  # местоимение-существительное
#        +'PRED',  # предикатив
#        +'PREP',  # предлог
#        +'CONJ',  # союз
#        +'PRCL',  # частица
#        +'INTJ',  # междометие
#}
#universal_tags={
#    +#ADJ: adjective
#    +#ADP: adposition
#    +#ADV: adverb
#    +#CCONJ: coordinating conjunction
#    +#DET: determiner
#    +#INTJ: interjection
#    +#NOUN: noun
#    +#NUM: numeral
#    +#PART: particle
#    +#PRON: pronoun
#    +#VERB: verb
#}

#Dict for tranformation pymorphy2 tags into universal POS tags

transform_dict = {'NOUN':'NOUN',
                  'ADJF':'ADJ',
                  'ADJS':'ADJ',
                  'PREP':'ADP',
                  'ADVB':'ADV',
                  'VERB':'VERB',
                  'INFN':'VERB',
                  'CONJ':'CCONJ',
                  'NPRO':'PRON',
                  'INTJ':'INTJ',
                  'PRCL':'PART',
                  'NUMR':'NUM',
                  'PRTF':'ADJ',
                  'PRTS':'ADJ',
                  'GRND':'ADJ',
                  'PRED':'ADJ'}

for index,row in data_df.iterrows():
    delete_list = list()
    #print('new comment')
    comment = data_df['text'].loc[index]
    for i in range(len(comment) - 1):
        print('word: ' + comment[i])
        tag = comment[i].split('_')[len(comment[i].split('_'))-1]

        print('old tag: ' + tag)
        if (tag == 'None'):
            delete_list.append(comment[i])
        else:
            tag = transform_dict.get(tag)
            print('new tag: ' + tag)
            comment[i] = comment[i].split('_')[0] + '_' + tag
    for delete_value in delete_list:
        comment.remove(delete_value)



word: ага_INTJ
old tag: INTJ
new tag: INTJ
word: держать_INFN
old tag: INFN
new tag: VERB
word: карман_NOUN
old tag: NOUN
new tag: NOUN
word: id41764449_None
old tag: None
word: али_CONJ
old tag: CONJ
new tag: CCONJ
word: вариант_NOUN
old tag: NOUN
new tag: NOUN
word: улететь_INFN
old tag: INFN
new tag: VERB
word: венера_NOUN
old tag: NOUN
new tag: NOUN
word: фиговый_ADJF
old tag: ADJF
new tag: ADJ
word: пациент_NOUN
old tag: NOUN
new tag: NOUN
word: либо_CONJ
old tag: CONJ
new tag: CCONJ
word: живой_NOUN
old tag: NOUN
new tag: NOUN
word: либо_CONJ
old tag: CONJ
new tag: CCONJ
word: мёртвый_NOUN
old tag: NOUN
new tag: NOUN
word: буратино_NOUN
old tag: NOUN
new tag: NOUN
word: читать_INFN
old tag: INFN
new tag: VERB
word: id176345518_None
old tag: None
word: валерий_NOUN
old tag: NOUN
new tag: NOUN
word: это_PRCL
old tag: PRCL
new tag: PART
word: аналитика_NOUN
old tag: NOUN
new tag: NOUN
word: чей_ADJF
old tag: ADJF
new tag: ADJ
word: реальность_NOUN
old tag: NOUN
new tag: NOUN
word: в

old tag: INFN
new tag: VERB
word: позитивный_ADJF
old tag: ADJF
new tag: ADJ
word: образ_NOUN
old tag: NOUN
new tag: NOUN
word: наш_ADJF
old tag: ADJF
new tag: ADJ
word: страна_NOUN
old tag: NOUN
new tag: NOUN
word: id223946369_None
old tag: None
word: михаил_NOUN
old tag: NOUN
new tag: NOUN
word: интересно_ADVB
old tag: ADVB
new tag: ADV
word: свой_ADJF
old tag: ADJF
new tag: ADJ
word: небесный_ADJF
old tag: ADJF
new tag: ADJ
word: id153889129_None
old tag: None
word: максим_NOUN
old tag: NOUN
new tag: NOUN
word: записываться_INFN
old tag: INFN
new tag: VERB
word: ряд_NOUN
old tag: NOUN
new tag: NOUN
word: навальнить_INFN
old tag: INFN
new tag: VERB
word: смочь_INFN
old tag: INFN
new tag: VERB
word: попасть_INFN
old tag: INFN
new tag: VERB
word: ряд_NOUN
old tag: NOUN
new tag: NOUN
word: тупорылый_ADJF
old tag: ADJF
new tag: ADJ
word: id4681388_None
old tag: None
word: михаил_NOUN
old tag: NOUN
new tag: NOUN
word: навальный_ADJF
old tag: ADJF
new tag: ADJ
word: любить_INFN
old tag: IN

old tag: INFN
new tag: VERB
word: давно_ADVB
old tag: ADVB
new tag: ADV
word: пора_NOUN
old tag: NOUN
new tag: NOUN
word: понять_INFN
old tag: INFN
new tag: VERB
word: япония_NOUN
old tag: NOUN
new tag: NOUN
word: являться_INFN
old tag: INFN
new tag: VERB
word: самостоятельный_ADJF
old tag: ADJF
new tag: ADJ
word: государство_NOUN
old tag: NOUN
new tag: NOUN
word: находиться_INFN
old tag: INFN
new tag: VERB
word: оккупация_NOUN
old tag: NOUN
new tag: NOUN
word: сша_NOUN
old tag: NOUN
new tag: NOUN
word: попытка_NOUN
old tag: NOUN
new tag: NOUN
word: вести_INFN
old tag: INFN
new tag: VERB
word: япония_NOUN
old tag: NOUN
new tag: NOUN
word: переговоры_NOUN
old tag: NOUN
new tag: NOUN
word: субъект_NOUN
old tag: NOUN
new tag: NOUN
word: международный_ADJF
old tag: ADJF
new tag: ADJ
word: право_NOUN
old tag: NOUN
new tag: NOUN
word: быть_INFN
old tag: INFN
new tag: VERB
word: что_CONJ
old tag: CONJ
new tag: CCONJ
word: пока_ADVB
old tag: ADVB
new tag: ADV
word: обрести_INFN
old tag: INFN
n

new tag: NOUN
word: терракт_NOUN
old tag: NOUN
new tag: NOUN
word: линия_NOUN
old tag: NOUN
new tag: NOUN
word: лэп_NOUN
old tag: NOUN
new tag: NOUN
word: лемур_NOUN
old tag: NOUN
new tag: NOUN
word: ислям_NOUN
old tag: NOUN
new tag: NOUN
word: чубаров_NOUN
old tag: NOUN
new tag: NOUN
word: шпион_NOUN
old tag: NOUN
new tag: NOUN
word: русофобствовать_INFN
old tag: INFN
new tag: VERB
word: историк_NOUN
old tag: NOUN
new tag: NOUN
word: прочий_ADJF
old tag: ADJF
new tag: ADJ
word: прекрасный_ADJF
old tag: ADJF
new tag: ADJ
word: редкостный_ADJF
old tag: ADJF
new tag: ADJ
word: бред_NOUN
old tag: NOUN
new tag: NOUN
word: написать_INFN
old tag: INFN
new tag: VERB
word: журналистка_NOUN
old tag: NOUN
new tag: NOUN
word: алиса_NOUN
old tag: NOUN
new tag: NOUN
word: смешать_INFN
old tag: INFN
new tag: VERB
word: куча_NOUN
old tag: NOUN
new tag: NOUN
word: репресия_NOUN
old tag: NOUN
new tag: NOUN
word: 30_None
old tag: None
word: х_None
old tag: None
word: 50_None
old tag: None
word: х_None
o

old tag: INFN
new tag: VERB
word: ненужный_ADJF
old tag: ADJF
new tag: ADJ
word: паритет_NOUN
old tag: NOUN
new tag: NOUN
word: танк_NOUN
old tag: NOUN
new tag: NOUN
word: авианосец_NOUN
old tag: NOUN
new tag: NOUN
word: прочий_ADJF
old tag: ADJF
new tag: ADJ
word: лабуд_NOUN
old tag: NOUN
new tag: NOUN
word: нацелить_INFN
old tag: INFN
new tag: VERB
word: калибр_NOUN
old tag: NOUN
new tag: NOUN
word: сидеть_INFN
old tag: INFN
new tag: VERB
word: курить_INFN
old tag: INFN
new tag: VERB
word: id77554707_None
old tag: None
word: сергей_NOUN
old tag: NOUN
new tag: NOUN
word: кэп_None
old tag: None
word: мы_NPRO
old tag: NPRO
new tag: PRON
word: бороться_INFN
old tag: INFN
new tag: VERB
word: игил_NOUN
old tag: NOUN
new tag: NOUN
word: защищать_INFN
old tag: INFN
new tag: VERB
word: наш_ADJF
old tag: ADJF
new tag: ADJ
word: вложение_NOUN
old tag: NOUN
new tag: NOUN
word: тот_ADJF
old tag: ADJF
new tag: ADJ
word: ливия_NOUN
old tag: NOUN
new tag: NOUN
word: помогать_INFN
old tag: INFN
new t

...

new tag: ADJ
word: поляк_NOUN
old tag: NOUN
new tag: NOUN
word: молодец_NOUN
old tag: NOUN
new tag: NOUN
word: приписывать_INFN
old tag: INFN
new tag: VERB
word: свой_ADJF
old tag: ADJF
new tag: ADJ
word: народ_NOUN
old tag: NOUN
new tag: NOUN
word: великий_ADJF
old tag: ADJF
new tag: ADJ
word: изобретение_NOUN
old tag: NOUN
new tag: NOUN
word: поляк_NOUN
old tag: NOUN
new tag: NOUN
word: балтийский_ADJF
old tag: ADJF
new tag: ADJ
word: мор_NOUN
old tag: NOUN
new tag: NOUN
word: выкопать_INFN
old tag: INFN
new tag: VERB
word: луна_NOUN
old tag: NOUN
new tag: NOUN
word: амер_NOUN
old tag: NOUN
new tag: NOUN
word: слетать_INFN
old tag: INFN
new tag: VERB
word: id257183006_None
old tag: None
word: андрей_NOUN
old tag: NOUN
new tag: NOUN
word: поляк_NOUN
old tag: NOUN
new tag: NOUN
word: побывать_INFN
old tag: INFN
new tag: VERB
word: луна_NOUN
old tag: NOUN
new tag: NOUN
word: соображение_NOUN
old tag: NOUN
new tag: NOUN
word: безопасность_NOUN
old tag: NOUN
new tag: NOUN
word: это_PRCL
o

old tag: INFN
new tag: VERB
word: любить_INFN
old tag: INFN
new tag: VERB
word: родина_NOUN
old tag: NOUN
new tag: NOUN
word: ненавидеть_INFN
old tag: INFN
new tag: VERB
word: id114037238_None
old tag: None
word: сергей_NOUN
old tag: NOUN
new tag: NOUN
word: учить_INFN
old tag: INFN
new tag: VERB
word: кричалка_NOUN
old tag: NOUN
new tag: NOUN
word: москаляк_NOUN
old tag: NOUN
new tag: NOUN
word: хиляк_NOUN
old tag: NOUN
new tag: NOUN
word: лагер_NOUN
old tag: NOUN
new tag: NOUN
word: азовец_NOUN
old tag: NOUN
new tag: NOUN
word: учить_INFN
old tag: INFN
new tag: VERB
word: id274125999_None
old tag: None
word: дзета_NOUN
old tag: NOUN
new tag: NOUN
word: мобильный_ADJF
old tag: ADJF
new tag: ADJ
word: интернет_NOUN
old tag: NOUN
new tag: NOUN
word: искать_INFN
old tag: INFN
new tag: VERB
word: id114037238_None
old tag: None
word: сергей_NOUN
old tag: NOUN
new tag: NOUN
word: мобильный_ADJF
old tag: ADJF
new tag: ADJ
word: интернет_NOUN
old tag: NOUN
new tag: NOUN
word: начаться_INFN
ol

word: это_PRCL
old tag: PRCL
new tag: PART
word: особенно_ADVB
old tag: ADVB
new tag: ADV
word: современный_ADJF
old tag: ADJF
new tag: ADJ
word: id350530155_None
old tag: None
word: виктор_NOUN
old tag: NOUN
new tag: NOUN
word: факт_NOUN
old tag: NOUN
new tag: NOUN
word: образование_NOUN
old tag: NOUN
new tag: NOUN
word: культура_NOUN
old tag: NOUN
new tag: NOUN
word: id241588771_None
old tag: None
word: алексей_NOUN
old tag: NOUN
new tag: NOUN
word: id350530155_None
old tag: None
word: виктор_NOUN
old tag: NOUN
new tag: NOUN
word: просто_PRCL
old tag: PRCL
new tag: PART
word: попытаться_INFN
old tag: INFN
new tag: VERB
word: оспорить_INFN
old tag: INFN
new tag: VERB
word: просто_PRCL
old tag: PRCL
new tag: PART
word: изложить_INFN
old tag: INFN
new tag: VERB
word: свой_ADJF
old tag: ADJF
new tag: ADJ
word: взгляд_NOUN
old tag: NOUN
new tag: NOUN
word: понятие_NOUN
old tag: NOUN
new tag: NOUN
word: слово_NOUN
old tag: NOUN
new tag: NOUN
word: сапиосексуал_NOUN
old tag: NOUN
new tag: N

word: 8_None
old tag: None
word: грн_None
old tag: None
word: доллар_NOUN
old tag: NOUN
new tag: NOUN
word: смочь_INFN
old tag: INFN
new tag: VERB
word: назвать_INFN
old tag: INFN
new tag: VERB
word: 2_None
old tag: None
word: достижение_NOUN
old tag: NOUN
new tag: NOUN
word: мудать_INFN
old tag: INFN
new tag: VERB
word: 1_None
old tag: None
word: свергнуть_INFN
old tag: INFN
new tag: VERB
word: пахан_NOUN
old tag: NOUN
new tag: NOUN
word: вор_NOUN
old tag: NOUN
new tag: NOUN
word: который_ADJF
old tag: ADJF
new tag: ADJ
word: сравнение_NOUN
old tag: NOUN
new tag: NOUN
word: нынешний_ADJF
old tag: ADJF
new tag: ADJ
word: кодла_NOUN
old tag: NOUN
new tag: NOUN
word: просто_PRCL
old tag: PRCL
new tag: PART
word: святой_ADJF
old tag: ADJF
new tag: ADJ
word: благотворитель_NOUN
old tag: NOUN
new tag: NOUN
word: бессеребренник_NOUN
old tag: NOUN
new tag: NOUN
word: время_NOUN
old tag: NOUN
new tag: NOUN
word: который_ADJF
old tag: ADJF
new tag: ADJ
word: большинство_NOUN
old tag: NOUN
new t

word: дозвук_NOUN
old tag: NOUN
new tag: NOUN
word: анализ_NOUN
old tag: NOUN
new tag: NOUN
word: крайний_ADJF
old tag: ADJF
new tag: ADJ
word: полёт_NOUN
old tag: NOUN
new tag: NOUN
word: складываться_INFN
old tag: INFN
new tag: VERB
word: впечатление_NOUN
old tag: NOUN
new tag: NOUN
word: определённый_ADJF
old tag: ADJF
new tag: ADJ
word: остаток_NOUN
old tag: NOUN
new tag: NOUN
word: топливо_NOUN
old tag: NOUN
new tag: NOUN
word: определённый_ADJF
old tag: ADJF
new tag: ADJ
word: высота_NOUN
old tag: NOUN
new tag: NOUN
word: самолёт_NOUN
old tag: NOUN
new tag: NOUN
word: лететь_INFN
old tag: INFN
new tag: VERB
word: сверхзвук_NOUN
old tag: NOUN
new tag: NOUN
word: режим_NOUN
old tag: NOUN
new tag: NOUN
word: работа_NOUN
old tag: NOUN
new tag: NOUN
word: двигатель_NOUN
old tag: NOUN
new tag: NOUN
word: id69193508_None
old tag: None
word: тарный_ADJF
old tag: ADJF
new tag: ADJ
word: эпр_None
old tag: None
word: ф_None
old tag: None
word: 22_None
old tag: None
word: дело_NOUN
old tag: 

In [4]:
import gensim
import codecs
from gensim.models.wrappers import FastText
import gensim.downloader as api

#FastText.load_model('cc.ru.300')
model = api.load("word2vec-ruscorpora-300")

#w2v_file = codecs.open('cc.ru.300.bin.gz', encoding='utf-8')
#model = gensim.models.KeyedVectors.load_word2vec_format(w2v_file, binary=True)  # or binary=False if the model is not compressed

D:\Anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
def create_average_vec(doc, vec_dim, ft_model):
    average = np.zeros((vec_dim,), dtype='float32')
    num_words = 0.
    for word in doc:
        if word in ft_model.wv.vocab:
            average = np.add(average, ft_model[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

word_counter = 0
model_counter = 0
for index,row in data_df.iterrows():
    comment = data_df['text'].loc[index]
    for word in comment:
        word_counter += 1
        if (word in model.vocab):
            model_counter += 1
print('All the words in data after preprocessing: ' + str(word_counter))
print('Words in model vacabulary: ' + str(model_counter))

All the words in data after preprocessing: 47792
Words in model vacabulary: 36163


In [6]:
import numpy as np
vector_data = list()


for index,row in data_df.iterrows(): 
    comment = data_df['text'].loc[index]
    comment_vector = np.zeros((300,), dtype='float32')
    words = 0
    for word in comment:
        if (word in model.vocab):
            comment_vector = np.add(comment_vector,model[word])
            words += 1
    if (words != 0):
        comment_vector = np.divide(comment_vector,words)
    vector_data.append(comment_vector)

In [13]:
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.utils import np_utils
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

label_list = list()
for intent in data_df['Intent']:
    label_list.append(intent)

# 4. Load pre-shuffled MNIST data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(vector_data, label_list, test_size=0.2, random_state = 42, stratify = label_list)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train) 
#y_test = np.array(y_test)

# 5. Preprocess input data
#X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
#X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
#X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
#X_train /= 255
#X_test /= 255
 
# 6. Preprocess class labels
#Y_train = np_utils.to_categorical(y_train, 25)
#Y_test = np_utils.to_categorical(y_test, 25)

encoder = LabelBinarizer()
encoder.fit(label_list)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# 7. Define model architecture
model = Sequential()
 
model.add(Conv1D(128, 5, activation='relu',input_dim = 300))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))
model.add(Dense(128, activation='relu'))
model.add(Dense(25, activation='softmax'))

model.summary()         
#model.add(Conv1D(128, 2, activation='relu',input_dim = 300, padding = 'same'))
#model.add(MaxPooling1D(1))
#model.add(Conv1D(128, 2, activation='relu'))
#model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu'))
#model.add(Dense(25, activation='softmax'))
#model.summary()

#model.compile(loss='categorical_crossentropy',
#                      optimizer='rmsprop',
#                      metrics=['acc'])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
          
# 9. Fit model on training data
model.fit(X_train, y_train, epochs=10, verbose=1)
 
# 10. Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 5, activation="relu", input_shape=(None, 300...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, None, 128)         192128    
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
dense_7 (Dense)              (None, None, 128)         16512     
__________

ValueError: Error when checking target: expected dense_8 to have 3 dimensions, but got array with shape (2928, 25)

In [20]:
print(len(X_test)+len(X_train))
print(len(y_test))


3661
3661


In [27]:
print(data_df['Intent'][35])

Щ


In [43]:
tokens = ['ага','держи','карман','шире']
for i in range(len(tokens)):
    print(tokens[i])
    tokens[i] = morph.parse(tokens[i])[0].normal_form
    t = morph.parse(tokens[i])[0]
    tokens[i] = tokens[i] + '_' + str(t.tag.POS)
    print(tokens[i])
print(tokens)


tokens = ['ага','держи','карман','шире']
for token in tokens:
    print(token)
    token = morph.parse(token)[0].normal_form
    t = morph.parse(token)[0]
    token = token+'_'+str(t.tag.POS)
    print(token)
print(tokens)

ага
ага_INTJ
держи
держать_INFN
карман
карман_NOUN
шире
шире_NOUN
['ага_INTJ', 'держать_INFN', 'карман_NOUN', 'шире_NOUN']
ага
ага_INTJ
держи
держать_INFN
карман
карман_NOUN
шире
шире_NOUN
['ага', 'держи', 'карман', 'шире']


In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

word = 'дома'
word = morph.parse(word)[0].normal_form
t = morph.parse(word)[0]
word = word+'_'+str(t.tag.POS)

In [32]:
vector_data[0]

array([-2.21383590e-02,  9.23040230e-03,  4.93700095e-02,  3.90502401e-02,
       -6.40397668e-02, -5.30002415e-02,  4.67836894e-02, -4.85614082e-03,
        4.02406836e-03, -9.79618821e-03,  2.84713488e-02,  9.16718785e-03,
        1.10953590e-02, -3.90402414e-02,  1.21535128e-02, -8.35934207e-02,
        7.45098069e-02,  1.62404403e-03,  1.74726788e-02,  3.05266287e-02,
        5.99543266e-02, -2.14239806e-02, -1.49271898e-02, -1.01301707e-02,
        1.79782417e-02, -2.90299635e-02,  3.24524790e-02, -2.79778410e-02,
       -2.37032931e-04, -5.76303340e-02,  4.24205773e-02,  6.51675323e-03,
        6.55684322e-02, -5.64103164e-02,  7.23182922e-03,  4.41771112e-02,
        4.30161357e-02, -1.89267937e-02, -4.42381762e-02,  2.29172464e-02,
        1.28449202e-02, -1.86000392e-02,  1.11480644e-02, -3.29818800e-02,
       -5.81772774e-02,  1.06284674e-03,  7.04645813e-02,  2.48348117e-02,
        7.25177452e-02, -2.24636495e-02,  9.71535686e-03,  4.66136783e-02,
       -1.86466146e-02, -

In [91]:
data_df['Intent'].unique()

array(['Ц', 'Н', 'Ш', 'Ж', 'И', 'Ч', 'Т', 'Щ', 'А', 'Г', 'У', 'Х', 'Б',
       'Д', 'Л', 'В', 'Е', 'П', 'К', 'С', 'М', 'Ф', 'З', 'Р', 'О'],
      dtype=object)

In [37]:
!pip install pymorphy2

  Running setup.py bdist_wheel for docopt: started
  Running setup.py bdist_wheel for docopt: finished with status 'done'
  Stored in directory: C:\Users\Tamir\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
distributed 1.20.2 requires msgpack-python, which is not installed.
tensorflow 1.11.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
pyowm 2.8.0 has requirement requests<2.19,>=2.18.2, but you'll have requests 2.19.1 which is incompatible.
copydoc 1.0.9 has requirement beautifulsoup4==4.4.1, but you'll have beautifulsoup4 4.6.3 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
